In [ ]:
!pip install bitsandbytes peft trl

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'


from datasets import load_dataset
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor, Qwen3VLProcessor, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
system_message = """You are a very advanced agent that is specialiezed on analyzing and interpreting images and text. Your task is to process images to understand if the content provided to you is safe or not for individuals. Please keep in mind the safety of others while categorizing images as safe or unsafe"""

def format_data(sample):
    return [
        {
            'role': 'system',
            'content': [{'type': 'text', 'text': system_message}],
        },
        {
            'role': 'user',
            'content': [
                {
                    'type': 'image',
                    'image': sample['image'],
                },
                {
                    'type': 'text',
                    'text': sample['text'],
                }
            ]
        },
        {
            'role': 'assistant',
            'content': [{'type': 'text', 'text': sample['safety_label']}]
        }
    ]

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

login(user_secrets.get_secret("hf_key"))

dataset_id = "yiting/UnsafeBench"
train_dataset, test_dataset = load_dataset(dataset_id, split=['train[:1%]', 'test[:2%]'])

In [ ]:
train_data = [format_data(sample) for sample in train_dataset]
test_data = [format_data(sample) for sample in test_dataset]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

MODEL_ID = "Qwen/Qwen3-VL-2B-Instruct"

In [ ]:
def load_model():
    if device == 'cuda':
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.bfloat16
        )
    
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            MODEL_ID,
            device_map='auto',
            quantization_config=bnb_config
        )
    else:
        model = Qwen3VLForConditionalGeneration.from_pretrained(
            MODEL_ID
        )
    
    processor = Qwen3VLProcessor.from_pretrained(MODEL_ID)
    processor.tokenizer.padding_size = 'right'

    return model, processor

ft_model, ft_processor = load_model()
base_model, base_processor = load_model()

In [ ]:
print(f'num of params before loading adapters: {ft_model.num_parameters()}')
ft_model.load_adapter('/kaggle/input/fine-tuned-qwen3vl/pytorch/default/1')
print(f'num of params before loading adapters: {ft_model.num_parameters()}')

In [ ]:
MAX_SEQ_LEN = 256

In [ ]:
def text_generator(sample_data, model, processor):
    text = processor.apply_chat_template(
        sample_data[:2], tokenize=False, add_generation_prompt=True
    )

    image_inputs = sample_data[1]['content'][0]['image']
    inputs = processor(
        text=[text],
        images= image_inputs,
        return_tensors='pt'
    )
    inputs = inputs.to(device)

    generated_ids = model.generate(**inputs, max_new_tokens=MAX_SEQ_LEN)

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )
    del inputs
    actual_answer = sample_data[2]['content'][0]['text']
    return output_text[0], actual_answer

In [ ]:
from IPython.display import display

print(test_data[30])
display(test_data[30][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[30], base_model, base_processor)

In [ ]:
text_generator(test_data[30], ft_model, ft_processor)

In [ ]:
print(test_data[31])
display(test_data[31][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[31], base_model, base_processor)

In [ ]:
text_generator(test_data[31], ft_model, ft_processor)

In [ ]:
print(test_data[32])
display(test_data[32][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[32], base_model, base_processor)

In [ ]:
text_generator(test_data[32], ft_model, ft_processor)

In [ ]:
print(test_data[33])
display(test_data[33][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[33], base_model, base_processor)

In [ ]:
text_generator(test_data[33], ft_model, ft_processor)

In [ ]:
print(test_data[34])
display(test_data[34][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[34], base_model, base_processor)

In [ ]:
text_generator(test_data[34], ft_model, ft_processor)

In [ ]:
print(test_data[35])
display(test_data[35][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[35], base_model, base_processor)

In [ ]:
text_generator(test_data[35], ft_model, ft_processor)

In [ ]:
print(test_data[36])
display(test_data[36][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[36], base_model, base_processor)

In [ ]:
text_generator(test_data[36], ft_model, ft_processor)

In [ ]:
print(test_data[37])
display(test_data[37][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[37], base_model, base_processor)

In [ ]:
text_generator(test_data[37], ft_model, ft_processor)

In [ ]:
print(test_data[38])
display(test_data[38][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[38], base_model, base_processor)

In [ ]:
text_generator(test_data[38], ft_model, ft_processor)

In [ ]:
print(test_data[39])
display(test_data[39][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[39], base_model, base_processor)

In [ ]:
text_generator(test_data[39], ft_model, ft_processor)

In [ ]:
print(test_data[40])
display(test_data[40][1]['content'][0]['image'])

In [ ]:
text_generator(test_data[40], base_model, base_processor)

In [ ]:
text_generator(test_data[40], ft_model, ft_processor)